# RNU4-2
[ReNU syndrome (RENU)](https://omim.org/entry/620851) is caused by heterozygous mutation in the RNU4-2 gene.

RNU4-2 is one of two U4 snRNA genes.

[Chen Y, et al. (2024)](https://pubmed.ncbi.nlm.nih.gov/38991538/) report:
> The n.64_65insT variant is one of six single base insertions that we observe in the 18 bp critical region in individuals with NDD, in a total of 100 individuals across cohorts. By contrast, single base insertions are very rare in population cohorts. Although we do also observe some SNVs in this region in individuals with NDD, our initial data suggest these SNVs may result in a milder phenotype. However, given this observation is based on only four fully phenotyped individuals, it needs to be confirmed in larger cohorts. 

In [1]:
import hpotk
import gpsea

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2024-08-13')
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-08-13
Using gpsea version 0.7.1


# RNU4-2
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript for RNU4-2. RNU4-2 is non-coding and thus there is no protein identifier.

In [2]:
gene_symbol = 'RNU4-2'
mane_tx_id = 'NR_003137.3'

In [4]:
from ppktstore.registry import configure_phenopacket_registry
from gpsea.preprocessing import load_phenopackets
from gpsea.preprocessing import configure_caching_cohort_creator

phenopacket_store_release = '0.1.21'  # Update, if necessary
registry = configure_phenopacket_registry()

with registry.open_phenopacket_store(release=phenopacket_store_release) as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(gene_symbol))

cohort_creator = configure_caching_cohort_creator(hpo)

cohort, qc_results = load_phenopackets(phenopackets, cohort_creator)  
qc_results.summarize()

Individuals Processed: 100%|██████████| 61/61 [00:03<00:00, 17.90individuals/s]
Validated under permissive policy
No errors or warnings were found


In [5]:
from gpsea.view import CohortViewer
viewer = CohortViewer(hpo)
viewer.process(cohort=cohort, transcript_id=mane_tx_id)

HPO Term,ID,Seen in n individuals
Absent speech,HP:0001344,49
Intellectual disability,HP:0001249,48
Delayed ability to walk,HP:0031936,46
Seizure,HP:0001250,45
Hypotonia,HP:0001252,42
Short stature,HP:0004322,38
Failure to thrive,HP:0001508,37
Severe global developmental delay,HP:0011344,35
Constipation,HP:0002019,35
Strabismus,HP:0000486,32


# Genotype phenotype correlation analysis

In [6]:
from gpsea.analysis.pcats import configure_hpo_term_analysis
analysis = configure_hpo_term_analysis(hpo)

from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest
pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)

In [8]:
from gpsea.analysis.predicate.genotype import VariantPredicates, monoallelic_predicate


is_n64_65insT = VariantPredicates.variant_key("12_120291839_120291839_T_TA") # n.64_65insT
is_b = VariantPredicates.variant_key("12_120291835_120291835_G_GT") #12_120291835_120291835_G_GT
is_c = VariantPredicates.variant_key("12_120291839_120291839_T_TC") # 12_120291839_120291839_T_TC
is_d = VariantPredicates.variant_key("12_120291826_120291826_T_TA") # 12_120291826_120291826_T_TA

is_insertion = is_n64_65insT | is_b | is_c | is_d 


is_n64_65insT_predicate = monoallelic_predicate(
    a_predicate=is_n64_65insT,
    b_predicate=~is_n64_65insT,
    a_label= "n.64_65insT", 
    b_label="other"
)

is_insertion_predicate = monoallelic_predicate(
    a_predicate=is_insertion,
    b_predicate=~is_insertion,
    a_label="insertion", 
    b_label="other"
)

In [9]:
n64_65insT_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=is_n64_65insT_predicate,
    pheno_predicates=pheno_predicates,
)
from gpsea.view import MtcStatsViewer

viewer = MtcStatsViewer()
viewer.process(n64_65insT_result)

Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.4,145
HMF03,Skipping term because of a child term with the same individual counts,10
HMF08,Skipping general term,92
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.4,234


In [10]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=n64_65insT_result)

Allele group                                       n.64_65insT  ...          
                                                         Count  ...  p values
Severe global developmental delay [HP:0011344]           33/38  ...  0.003469
Moderate global developmental delay [HP:0011343]          5/38  ...  0.003469
Absent speech [HP:0001344]                               45/51  ...  0.022273
Hypotonia [HP:0001252]                                   46/49  ...  0.030777
Hearing impairment [HP:0000365]                           6/38  ...  0.130678
Primary microcephaly [HP:0011451]                        19/34  ...  0.181764
Seizure [HP:0001250]                                     37/52  ...  0.182440
Constipation [HP:0002019]                                32/48  ...  0.241929
Delayed ability to walk [HP:0031936]                     40/41  ...  0.273050
Strabismus [HP:0000486]                                  29/49  ...  0.446515
Tented philtrum [HP:0011825]                             19/44  ...  0.449464
Intrauterine growth retardation [HP:0001511]             16/39  ...  0.633704
Secondary microcephaly [HP:0005484]                      15/43  ...  0.655735
Failure to thrive [HP:0001508]                           33/47  ...  0.665264
Growth delay [HP:0001510]                                38/38  ...  1.000000
Delayed gross motor development [HP:0002194]             40/41  ...  1.000000
Motor delay [HP:0001270]                                 50/50  ...  1.000000
Neurodevelopmental delay [HP:0012758]                    53/53  ...  1.000000
Neurodevelopmental abnormality [HP:0012759]              53/53  ...  1.000000
Feeding difficulties [HP:0011968]                        33/35  ...  1.000000
Abdominal symptom [HP:0011458]                           40/40  ...  1.000000
Short stature [HP:0004322]                               33/43  ...  1.000000
Global developmental delay [HP:0001263]                  53/53  ...  1.000000
Intellectual disability [HP:0001249]                     46/46  ...  1.000000
Aplasia/Hypoplasia involving the central nervou...       43/43  ...  1.000000
Morphological central nervous system abnormalit...       48/48  ...  1.000000
Nystagmus [HP:0000639]                                   22/44  ...  1.000000
Abnormal involuntary eye movements [HP:0012547]          22/22  ...  1.000000
Decreased body weight [HP:0004325]                       33/33  ...  1.000000
Microcephaly [HP:0000252]                                41/41  ...  1.000000
Abnormal axial skeleton morphology [HP:0009121]          46/46  ...  1.000000
Aplasia/Hypoplasia of the cerebrum [HP:0007364]          41/41  ...  1.000000
Abnormal cerebral morphology [HP:0002060]                44/44  ...  1.000000
Abnormal forebrain morphology [HP:0100547]               44/44  ...  1.000000
Abnormal brain morphology [HP:0012443]                   45/45  ...  1.000000
Abnormal speech pattern [HP:0002167]                     52/52  ...  1.000000
Abnormal communication [HP:0034434]                      52/52  ...  1.000000
Delayed speech and language development [HP:000...       52/52  ...  1.000000
Abnormality of the mouth [HP:0000153]                    36/36  ...  1.000000
Abnormal oral cavity morphology [HP:0000163]             36/36  ...  1.000000
Abnormal eyelid morphology [HP:0000492]                  29/29  ...  1.000000
Abnormal ocular adnexa morphology [HP:0030669]           29/29  ...  1.000000
Abnormal muscle tone [HP:0003808]                        47/47  ...  1.000000
Abnormal lip morphology [HP:0000159]                     29/29  ...  1.000000
Abnormal nasal morphology [HP:0005105]                   26/26  ...  1.000000
Abnormal conjugate eye movement [HP:0000549]             29/29  ...  1.000000
Gastroesophageal reflux [HP:0002020]                     23/48  ...  1.000000
Abnormal esophagus physiology [HP:0025270]               23/23  ...  1.000000
Abnormality of the philtrum [HP:0000288]                 23/23  ...  1.000000
Abnormal upper lip mor

In [11]:
insertion_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=is_insertion_predicate,
    pheno_predicates=pheno_predicates,
)
summarize_hpo_analysis(hpo=hpo, result=insertion_result)

Allele group                                       insertion  ...          
                                                       Count  ...  p values
Moderate global developmental delay [HP:0011343]        6/41  ...  0.001409
Severe global developmental delay [HP:0011344]         35/41  ...  0.001409
Absent speech [HP:0001344]                             47/54  ...  0.030294
High forehead [HP:0000348]                              3/56  ...  0.049231
Diabetes insipidus [HP:0000873]                         9/48  ...  0.070212
Hypotonia [HP:0001252]                                 48/52  ...  0.080878
Failure to thrive [HP:0001508]                         36/50  ...  0.087083
Constipation [HP:0002019]                              34/51  ...  0.131196
Delayed ability to walk [HP:0031936]                   43/44  ...  0.161348
Short stature [HP:0004322]                             36/46  ...  0.239917
Seizure [HP:0001250]                                   40/55  ...  0.317977
Primary microcephaly [HP:0011451]                      19/36  ...  0.604990
Nystagmus [HP:0000639]                                 24/47  ...  0.609844
Secondary microcephaly [HP:0005484]                    16/45  ...  0.618126
Growth delay [HP:0001510]                              41/41  ...  1.000000
Delayed gross motor development [HP:0002194]           43/44  ...  1.000000
Motor delay [HP:0001270]                               53/53  ...  1.000000
Neurodevelopmental delay [HP:0012758]                  56/56  ...  1.000000
Neurodevelopmental abnormality [HP:0012759]            56/56  ...  1.000000
Feeding difficulties [HP:0011968]                      36/38  ...  1.000000
Abdominal symptom [HP:0011458]                         43/43  ...  1.000000
Global developmental delay [HP:0001263]                56/56  ...  1.000000
Intellectual disability [HP:0001249]                   49/49  ...  1.000000
Aplasia/Hypoplasia involving the central nervou...     45/45  ...  1.000000
Morphological central nervous system abnormalit...     50/50  ...  1.000000
Abnormal involuntary eye movements [HP:0012547]        24/24  ...  1.000000
Decreased body weight [HP:0004325]                     36/36  ...  1.000000
Microcephaly [HP:0000252]                              43/43  ...  1.000000
Abnormal axial skeleton morphology [HP:0009121]        49/49  ...  1.000000
Aplasia/Hypoplasia of the cerebrum [HP:0007364]        43/43  ...  1.000000
Abnormal cerebral morphology [HP:0002060]              46/46  ...  1.000000
Abnormal forebrain morphology [HP:0100547]             46/46  ...  1.000000
Abnormal brain morphology [HP:0012443]                 47/47  ...  1.000000
Abnormal speech pattern [HP:0002167]                   54/54  ...  1.000000
Abnormal communication [HP:0034434]                    54/54  ...  1.000000
Delayed speech and language development [HP:000...     54/54  ...  1.000000
Abnormality of the mouth [HP:0000153]                  37/37  ...  1.000000
Abnormal oral cavity morphology [HP:0000163]           37/37  ...  1.000000
Abnormal eyelid morphology [HP:0000492]                30/30  ...  1.000000
Abnormal ocular adnexa morphology [HP:0030669]         30/30  ...  1.000000
Abnormal muscle tone [HP:0003808]                      49/49  ...  1.000000
Abnormal lip morphology [HP:0000159]                   30/30  ...  1.000000
Abnormal nasal morphology [HP:0005105]                 27/27  ...  1.000000
Strabismus [HP:0000486]                                30/52  ...  1.000000
Abnormal conjugate eye movement [HP:0000549]           30/30  ...  1.000000
Gastroesophageal reflux [HP:0002020]                   24/51  ...  1.000000
Abnormal esophagus physiology [HP:0025270]             24/24  ...  1.000000
Tented philtrum [HP:0011825]                           19/47  ...  1.000000
Abnormality of the philtrum [HP:0000288]               23/23  ...  1.000000
Abnormal upper lip morphology [HP:0000177]             23/23  ...  1.000000
Recurrent maladaptive behavior [HP:5200241]     

In [12]:
from gpsea.analysis.predicate.genotype import sex_predicate
mf_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=sex_predicate(),
    pheno_predicates=pheno_predicates,
)

summarize_hpo_analysis(hpo, mf_result)

Sex                                                FEMALE  ...          
                                                    Count  ...  p values
Reduced circulating growth hormone concentratio...   0/11  ...  0.005025
Feeding difficulties [HP:0011968]                   16/18  ...  0.196154
Gastroesophageal reflux [HP:0002020]                 9/24  ...  0.277773
Delayed gross motor development [HP:0002194]        19/20  ...  0.425532
Short stature [HP:0004322]                          18/22  ...  0.511611
Intrauterine growth retardation [HP:0001511]         8/17  ...  0.525863
Hypotonia [HP:0001252]                              24/26  ...  0.677940
Facial hypotonia [HP:0000297]                        4/10  ...  0.692449
Severe global developmental delay [HP:0011344]      15/20  ...  0.731035
Absent speech [HP:0001344]                          23/27  ...  0.741246
Primary microcephaly [HP:0011451]                    9/16  ...  0.747527
Failure to thrive [HP:0001508]                      15/23  ...  0.769407
Nystagmus [HP:0000639]                              10/22  ...  0.779349
Strabismus [HP:0000486]                             14/23  ...  0.784899
Growth delay [HP:0001510]                           21/21  ...  1.000000
Delayed ability to walk [HP:0031936]                19/20  ...  1.000000
Motor delay [HP:0001270]                            25/25  ...  1.000000
Neurodevelopmental delay [HP:0012758]               28/28  ...  1.000000
Neurodevelopmental abnormality [HP:0012759]         28/28  ...  1.000000
Abdominal symptom [HP:0011458]                      20/20  ...  1.000000
Global developmental delay [HP:0001263]             28/28  ...  1.000000
Intellectual disability [HP:0001249]                23/23  ...  1.000000
Aplasia/Hypoplasia involving the central nervou...  21/21  ...  1.000000
Morphological central nervous system abnormalit...  25/25  ...  1.000000
Abnormal involuntary eye movements [HP:0012547]     10/10  ...  1.000000
Decreased body weight [HP:0004325]                  15/15  ...  1.000000
Constipation [HP:0002019]                           15/23  ...  1.000000
Seizure [HP:0001250]                                20/27  ...  1.000000
Microcephaly [HP:0000252]                           20/20  ...  1.000000
Abnormal axial skeleton morphology [HP:0009121]     24/24  ...  1.000000
Aplasia/Hypoplasia of the cerebrum [HP:0007364]     20/20  ...  1.000000
Abnormal cerebral morphology [HP:0002060]           22/22  ...  1.000000
Abnormal forebrain morphology [HP:0100547]          22/22  ...  1.000000
Abnormal brain morphology [HP:0012443]              23/23  ...  1.000000
Abnormal speech pattern [HP:0002167]                27/27  ...  1.000000
Abnormal communication [HP:0034434]                 27/27  ...  1.000000
Delayed speech and language development [HP:000...  27/27  ...  1.000000
Abnormality of the mouth [HP:0000153]               19/19  ...  1.000000
Abnormal oral cavity morphology [HP:0000163]        19/19  ...  1.000000
Abnormal eyelid morphology [HP:0000492]             14/14  ...  1.000000
Abnormal ocular adnexa morphology [HP:0030669]      14/14  ...  1.000000
Abnormal muscle tone [HP:0003808]                   25/25  ...  1.000000
Abnormal lip morphology [HP:0000159]                16/16  ...  1.000000
Abnormal nasal morphology [HP:0005105]              10/10  ...  1.000000
Abnormal conjugate eye movement [HP:0000549]        14/14  ...  1.000000
Abnormal esophagus physiology [HP:0025270]            9/9  ...  1.000000
Tented philtrum [HP:0011825]                        11/26  ...  1.000000
Abnormality of the philtrum [HP:0000288]            14/14  ...  1.000000
Abnormal upper lip morphology [HP:0000177]          14/14  ...  1.000000
Recurrent maladaptive behavior [HP:5200241]         14/14  ...  1.000000
Atypical behavior [HP:0000708]                      15/15  ...  1.000000
Gait disturbance [HP:0001288]                       15/15  ...  1.000000

[52 rows x 6 columns]

# Summary

In [13]:
from gpseacs.report import GpseaAnalysisReport, GPAnalysisResultSummary


fet_results = (
    GPAnalysisResultSummary.from_multi(
        result=n64_65insT_result,
    ),
    GPAnalysisResultSummary.from_multi(
        result=insertion_result,
    ),
    GPAnalysisResultSummary.from_multi(
        result=mf_result
    )
)

caption = "No previous statistical analysis of correlations with PTPN11 missense variants identified in the medical literature."
report = GpseaAnalysisReport(name=gene_symbol, 
                             cohort=cohort, 
                             fet_results=fet_results,
                             gene_symbol=gene_symbol,
                             mane_tx_id=mane_tx_id,
                             mane_protein_id="",
                             caption=caption)

In [14]:
from gpseacs.report import GpseaNotebookSummarizer
summarizer = GpseaNotebookSummarizer(hpo=hpo, gpsea_version=gpsea.__version__)
summarizer.summarize_report(report=report)

In [15]:
summarizer.process_latex(report=report)

Output to ../../supplement/tex/RNU4-2_summary_draft.tex
